# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

101


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




In [8]:
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(sessionId int, itemInSession int, artist text, duration float, song text, PRIMARY KEY (sessionId, itemInSession))"

session.execute(query)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions (sessionid, iteminsession, artist, duration, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        #print(type(line[8]), type(line[3]), type(line[0]), type(line[5]), type(line[9]))
      

In [10]:
#commented line give the quality check result with checking parameters

#query_case_1 = "SELECT * FROM sessions "
query_case_1 = "SELECT artist, song, duration FROM sessions "
query_case_1 +="WHERE sessionid = 338 AND iteminsession = 4"


try:
    rows = session.execute(query_case_1)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.artist, row.song, row.duration, row.sessionid, row.iteminsession)
    print (row.artist, row.song, row.duration)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY (userid, sessionid, iteminsession))"

session.execute(query)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        #print(type(line[8]), type(line[3]), type(line[0]), type(line[5]), type(line[9]))
      

In [13]:
#commented line give the quality check result with checking parameters

#query_case_2 = "SELECT * FROM users "
query_case_2 = "SELECT artist, song, firstname, lastname FROM users "
query_case_2 +="WHERE userid = 10 AND sessionid = 182 "


try:
    rows = session.execute(query_case_2)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.artist, row.song, row.firstname, row.lastname, row.userid, row.sessionid, row.iteminsession)
    print (row.artist, row.song, row.firstname, row.lastname)
                    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(song text, userid int, sessionid int, firstname text, lastname text, PRIMARY KEY (song, userid, sessionid))"

session.execute(query)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs (song, userid, sessionid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        session.execute(query, (line[9], int(line[10]), int(line[8]), line[1], line[4]))
        #print(type(line[8]), type(line[3]), type(line[0]), type(line[5]), type(line[9]))
      

In [16]:
#commented line give the quality check result with checking parameters

#query_case_3 = "SELECT * FROM songs "
query_case_3 = "SELECT firstname, lastname FROM songs "
query_case_3 +="WHERE song = 'All Hands Against His Own'"


try:
    rows = session.execute(query_case_3)
except Exception as e:
    print(e)
    
for row in rows:
    #print (row.song, row.firstname, row.lastname,)
    print (row.firstname, row.lastname)

Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "drop table sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()